# RAG Experiments

### Importing Libraries and Loading Data

In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt

In [2]:
q_train = pd.read_csv("data/q_train_df.csv", sep="|")
q_test = pd.read_csv("data/q_test_df.csv", sep="|")
q_train.head()

,Question,Embedding
0,Which NFL team represented the AFC at Super Bo...,"[-0.018374031, -0.02516927, 0.008701543, -0.08..."
1,Which NFL team represented the NFC at Super Bo...,"[-0.0065157767, -0.01013894, -0.016297542, -0...."
2,Where did Super Bowl 50 take place?,"[-0.017278872, -0.0012698264, 0.00483831, -0.1..."
3,Which NFL team won Super Bowl 50?,"[-0.0070417165, 0.008862875, 0.03743299, -0.08..."
4,What color was used to emphasize the 50th anni...,"[-0.0697444, 0.030223556, 0.0025628584, -0.076..."


### Connect to DataBase

In [3]:
%pip install --upgrade --quiet  pgvector
%pip install --upgrade --quiet  langchain-openai
%pip install --upgrade --quiet  psycopg2-binary
%pip install --upgrade --quiet  tiktoken

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


In [4]:
# this is to import all necessary modules
import requests
import matplotlib.pyplot as plt
import openai, json, tiktoken, psycopg2, ast, pgvector, math, os, random

from langchain.vectorstores.pgvector import PGVector
from psycopg2.extras import execute_values
from sqlalchemy import create_engine
from time import time
#from pgvector.psycopg2 import register_vector
#%load_ext sql

In [5]:
conn_string = "host='localhost' dbname='imdb_top' user='postgres' password='test'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

In [14]:
cur = conn.cursor()
cur.execute("ROLLBACK")
conn.commit()

In [23]:
def get_article(vec, value=None):
    query = f"""
        SELECT content FROM (
            SELECT content, embedding <=> '{vec}' AS cos_sim
            FROM squad_articles
            ORDER BY cos_sim
            LIMIT 1
        ) AS subquery;
    """

    cur.execute(query)
    conn.commit()
    result = cur.fetchall()
    return result[0][0]

In [57]:
q_test["Question_Context"] = np.zeros(q_test.shape[0])
for i in range(q_test.shape[0]):
    context = get_article(q_test["Embedding"][i])
    q_test["Question_Context"][i] = q_test["Question"][i] + "\n\nYou may want to use the context:\n" + context
    if i % 100 == 0: print(i)
    

C:\Users\Abylay\AppData\Local\Temp\ipykernel_29580\3534837153.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q_test["Question_Context"][i] = q_test["Question"][i] + "\n\nYou may want to use the context:\n" + context


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100


In [58]:
q_test.head()

,Question,Embedding,Question_Context
0,Who became king in 1643?,"[-0.043172576, 0.06543647, 0.053647112, 0.0173...",Who became king in 1643?\n\nYou may want to us...
1,What was the first team Peyton Manning began p...,"[0.0154337315, 0.024577715, -0.019312672, -0.0...",What was the first team Peyton Manning began p...
2,What was East and Central Africa's economy boo...,"[-0.096133806, 0.05678508, 0.08325848, 0.03994...",What was East and Central Africa's economy boo...
3,How many buildings were razed by the Jacksonvi...,"[0.011810559, 0.02840623, 0.024501665, 0.00578...",How many buildings were razed by the Jacksonvi...
4,What year was the first manned flight with the...,"[-0.0056205806, 0.04971382, 0.030576339, -0.01...",What year was the first manned flight with the...


In [19]:
type(q_test["Question"][0])

str

In [25]:
np.zeros(q_test.shape[0])

array([0., 0., 0., ..., 0., 0., 0.])

In [30]:
context

'There were two kinds of X.25 networks. Some such as DATAPAC and TRANSPAC were initially implemented with an X.25 external interface. Some older networks such as TELENET and TYMNET were modified to provide a X.25 host interface in addition to older host connection schemes. DATAPAC was developed by Bell Northern Research which was a joint venture of Bell Canada (a common carrier) and Northern Telecom (a telecommunications equipment supplier). Northern Telecom sold several DATAPAC clones to foreign PTTs including the Deutsche Bundespost. X.75 and X.121 allowed the interconnection of national X.25 networks. A user or host could call a host on a foreign network by including the DNIC of the remote network as part of the destination address.[citation needed]'

In [37]:
q_test["Question_Context"][2]

"What was East and Central Africa's economy boosted by?\n\nYou may use the following text as context:\nEast and Central Africa's biggest economy has posted tremendous growth in the service sector, boosted by rapid expansion in telecommunication and financial activity over the last decade, and now[when?] contributes 62% of GDP. 22% of GDP still comes from the unreliable agricultural sector which employs 75% of the labour force (a consistent characteristic of under-developed economies that have not attained food security – an important catalyst of economic growth) A small portion of the population relies on food aid.[citation needed] Industry and manufacturing is the smallest sector, accounting for 16% of GDP. The service, industry and manufacturing sectors only employ 25% of the labour force but contribute 75% of GDP."

In [46]:
q_test.to_csv("data/q_context.csv", sep="|", index=False)

In [47]:
q_test.shape

(2114, 3)

In [48]:
q_test_load = pd.read_csv("data/q_context.csv", sep="|")

In [59]:
q_test_load.shape

(2114, 3)

In [60]:
q_train["Question_Context"] = np.zeros(q_train.shape[0])
for i in range(q_train.shape[0]):
    context = get_article(q_train["Embedding"][i])
    q_train["Question_Context"][i] = q_train["Question"][i] + "\n\nYou may want to use the context:\n" + context
    if i % 100 == 0: print(i)
    

C:\Users\Abylay\AppData\Local\Temp\ipykernel_29580\2442682239.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q_train["Question_Context"][i] = q_train["Question"][i] + "\n\nYou may want to use the context:\n" + context


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400


In [64]:
q_train.shape

(8456, 3)

In [65]:
q_train.to_csv("data/qc_train_df.csv", sep="|", index=False)

In [67]:
qc_train_df = pd.read_csv("data/qc_train_df.csv", sep="|")
qc_train_df.shape

(8456, 3)

In [53]:
a_test = np.load("data/answers_test.npy", allow_pickle=True)
a_test

array([list(['Louis XIV', 'Louis XIV', 'Louis XIV']),
       list(['Indianapolis Colts', 'the Indianapolis Colts', '1998']),
       list(['rapid expansion in telecommunication and financial activity', 'rapid expansion in telecommunication and financial activity', 'rapid expansion in telecommunication and financial activity']),
       ...,
       list(['ownership of private industries', 'technological superiority', 'technological superiority', 'ownership of private industries', 'technological superiority,']),
       list(['arbitrary graph', 'arbitrary', 'arbitrary']),
       list(['Internet2 officially retired Abilene and now refers to its new, higher capacity network as the Internet2 Network', 'Internet2 Network', 'Internet2 Network'])],
      dtype=object)

In [56]:
q_test_load["Question_Context"][10]

"How long was Warsaw the capital of the Polish-Lithuanian Commonwealth?\n\nYou may use the following text as context:\nWarsaw remained the capital of the Polish–Lithuanian Commonwealth until 1796, when it was annexed by the Kingdom of Prussia to become the capital of the province of South Prussia. Liberated by Napoleon's army in 1806, Warsaw was made the capital of the newly created Duchy of Warsaw. Following the Congress of Vienna of 1815, Warsaw became the centre of the Congress Poland, a constitutional monarchy under a personal union with Imperial Russia. The Royal University of Warsaw was established in 1816."